In [1]:
from qiskit.circuit.library import EfficientSU2
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_algorithms.optimizers import COBYLA

from circuit_knitting.cutting.gate_and_wire_cutting.algorithms.cut_vqe import CutVQE


# Use a list of H2 observables from the PySCF driver
H2_op_list = [('II', -1.0523732457728596),
             ('IZ', 0.39793742484317934),
             ('XX', 0.18093119978423144),
             ('ZI', -0.39793742484317934),
             ('ZZ', -0.011280104256235268)]

# Problem specification. Hamiltonian for H2 at 0.735A interatomic distance.
H2_op = SparsePauliOp.from_list(H2_op_list)

# Create an ansatz
ansatz = EfficientSU2(H2_op.num_qubits, reps=1)

# Decompose into individual gates
circuit = ansatz.decompose(reps=3)

circuit

In [2]:
# See how many parameters are in our ansatz
num_params = circuit.num_parameters
print(f'Num parameters: {num_params}')

# List out the observables
observables = H2_op.paulis
print(f'Observables: {observables}')

Num parameters: 8
Observables: ['II', 'IZ', 'XX', 'ZI', 'ZZ']


In [3]:
# Create a callback function that will be used to get VQE data during the run 
def callback(eval_count, parameters, mean, std):
    print(f"Round num: {eval_count}, energy: {mean}, parameters: {parameters}")

In [4]:
# Use the modified CutVQE to cut and run VQE for our problem
estimator = Estimator()
optimizer = COBYLA(maxiter=80)
vqe = CutVQE(estimator, circuit, optimizer, observables=observables, shots=2**12, max_subcircuit_width=1, max_cuts=9, num_subcircuits=[2], model='gurobi', num_samples=1500, callback=callback)
result = vqe.compute_minimum_eigenvalue(H2_op)
print(result)

Restricted license - for non-production use only - expires 2025-11-24
Set parameter TimeLimit to value 300
Set parameter Cutoff to value 1e+100
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.0.0 23A344)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 31 rows, 21 columns and 74 nonzeros
Model fingerprint: 0xb35f4f20
Variable types: 0 continuous, 21 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e-02, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 1.0000000
Presolve removed 31 rows and 21 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 1 

Optimal solution found (tolerance 1.00e-04)
Bes

MIP MODEL CUT EDGES:  [('q[0]0', 'q[1]0')]
Round num: 1, energy: -1.44374692363128, parameters: [ 2.3506968  -5.96849212  6.11149524 -1.60209118 -4.2106781   1.29950986
  0.63350955  1.82991057]
Round num: 2, energy: -1.5219893202414896, parameters: [ 3.3506968  -5.96849212  6.11149524 -1.60209118 -4.2106781   1.29950986
  0.63350955  1.82991057]
Round num: 3, energy: -1.5173661058588237, parameters: [ 3.3506968  -4.96849212  6.11149524 -1.60209118 -4.2106781   1.29950986
  0.63350955  1.82991057]
Round num: 4, energy: -1.3941760338799063, parameters: [ 3.3506968  -5.96849212  7.11149524 -1.60209118 -4.2106781   1.29950986
  0.63350955  1.82991057]
Round num: 5, energy: -1.5126907793718343, parameters: [ 3.3506968  -5.96849212  6.11149524 -0.60209118 -4.2106781   1.29950986
  0.63350955  1.82991057]
Round num: 6, energy: -1.7141576203718858, parameters: [ 3.3506968  -5.96849212  6.11149524 -1.60209118 -3.2106781   1.29950986
  0.63350955  1.82991057]
Round num: 7, energy: -1.7169947474

In [5]:
# Do the same thing to compare againt not cutting at all
estimator = Estimator()
optimizer = COBYLA(maxiter=80)
no_cut_vqe = VQE(ansatz=circuit, optimizer=optimizer, estimator=estimator, callback=callback)
no_cut_result = no_cut_vqe.compute_minimum_eigenvalue(H2_op)
print(no_cut_result)

Round num: 1, energy: -1.5829235863580422, parameters: [0.18174392152896424, 3.2622967900872286, -5.046784129015688, 0.19924163106112047, 2.0015373106993266, 3.2945957187324435, 5.146979993961116, -0.6719533787235372]
Round num: 2, energy: -1.2842281143715102, parameters: [1.1817439215289642, 3.2622967900872286, -5.046784129015688, 0.19924163106112047, 2.0015373106993266, 3.2945957187324435, 5.146979993961116, -0.6719533787235372]
Round num: 3, energy: -1.2838241100049284, parameters: [0.18174392152896424, 4.262296790087229, -5.046784129015688, 0.19924163106112047, 2.0015373106993266, 3.2945957187324435, 5.146979993961116, -0.6719533787235372]
Round num: 4, energy: -1.6090380857066495, parameters: [0.18174392152896424, 3.2622967900872286, -4.046784129015688, 0.19924163106112047, 2.0015373106993266, 3.2945957187324435, 5.146979993961116, -0.6719533787235372]
Round num: 5, energy: -1.6184867478074036, parameters: [0.18174392152896424, 3.2622967900872286, -4.046784129015688, 1.19924163106